### David Herrera
### Estid Lozano

In [4]:
import numpy as np
import openml as oml
from tqdm.keras import TqdmCallback

from sklearn import linear_model, svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Exercise 1

Show the following:

**1.1.** The weighted error rate $\varepsilon_t$ compute in line 7 of AdaBoost is always between 0 and 1.

**1.2.** The weight $\alpha_t$ of the classifier in round $t$ is always strictly positive

**1.3.** Suppose that the model $M_t$ miss-classified instance $x_i$ in iteration $t$.

**a)** The weight for the instance cannot decrease, i.e. $w^{t+1}_i \geq w^t_i$. That is, the weight vector computed in line 14 has for every misclassified instance a value that is not smaller as the value the vector $w^t$ had before. And show that it will strictly increase unless all instances were misclassified.

*Hint: Note that $w^{t−1}$ has a total sum of 1 when visiting the lines 10-14.*

**b)** The lower the weighted error rate over all the instances, the stronger the weight for instance $i$ will increase.

*Hint: Consider two error rates $\epsilon^a$ and $\epsilon^b$ with $\epsilon^a > \epsilon^b$ and observe the effect on $w^t_{i,a}<w^t_{i,b}$, where $w^t_{i,q}$ is the weight obtained when the overall error rate is $q$.*

# Exercise 2

**2.1.** Implement the AdaBoost algorithm and the Stacking algorithm

In [40]:
def AdaBoost(M, X, y):
    n = len(X)
    range_n = range(n)
    ones = np.repeat(1, n)
    W = np.array([np.repeat(1 / n, n)])
    for t, M_t in enumerate(M):
        i_t = np.random.randint(0, n, n) # random.choices(np.arange(n), weights=W[t], k=n)
        X_t, y_t = X[i_t], y[i_t]
        M_t.fit(X_t, y_t, verbose=0, callbacks=[TqdmCallback(verbose=1)]) # On D_t
        I = np.array([M_t.predict(X[i]) != y[i] for i in range_n]) # M_t(x_i) != y_i on D
        E_t = sum(W[t][I]) # Sum W_t[i] where I[i] is True
        if E_t == 0:
            break
        elif E_t < 0.5:
            a = (1 - E_t) / E_t
            alpha_t = np.log(a) # WHERE IS USED?
            W = W.vstack([W, (a * I) * W[t]]) # Multiplies W_t[i] by a
            W[-1] = W[-1] / (np.dot(ones, W[-1]))
    return M

In [ ]:
def Stacking(M, C, X, y):
    # Train base classifiers
    for M_t in M:
        M_t.fit(X, y, verbose=0, callbacks=[TqdmCallback(verbose=1)])
    # Train combiner model C on Z
    Z = np.array([[M_t.predict(x_i) for M_t in M] for x_i in X])
    C.fit(Z, y, verbose=0, callbacks=[TqdmCallback(verbose=1)])
    return [C] + M

**2.2.** Explore the openml.org platform and search for two datasets with two classes and at least 1000 instances.

Apply AdaBoost and Stacking to these two datasets and report performances and runtimes. Use AdaBoost with at least 5 different learners from the scikit-learn library.
For Stacking, use these learners in layer 1 and try each of them once as the learner to aggregate the predictions.

**2.3.** Interpret your results.